In [1]:
from Utility import df_ops, music_ops, vmd_ops, io_ops, cloud_ops, interpolate
import pandas as pd
import numpy as np
import pymeshio.common

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Don't wrap to multiple pages
pd.set_option('max_colwidth', None)  # No max column width


#df_ops.saveDfToFeather(vmd_ops.getDfFromVmdFileName("play\\play.vmd"), "play\\play.feather")

c:\Users\daryl\AppData\Local\Programs\Python\Python311\Lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# df = df_ops.loadDfFromFeather('play\\play.feather')
# df = vmd_ops.filterDataframeForValidBoneNames(df)
# vmd_ops.saveDfToVmdFile(df, 'play\\validBone.vmd')

In [12]:

def normalizePositionsToCenter(df):
  df_position_bones = df[df['name'].isin(['センター', '左足ＩＫ', '右足ＩＫ'])]
  complement = vmd_ops.getComplement(df)
  lastFrame = df_ops.getLastFrame(df_position_bones)
  centerBone = 'センター'
  footIKBones = ['左足ＩＫ', '右足ＩＫ']
  boneList = [centerBone] + footIKBones

  lastBoneList = {boneName: {"frameNumber": None, "position": None, "rotation": None} for boneName in boneList}
  for boneName in lastBoneList.keys(): #init lastbonelist
    lastBoneList[boneName] = interpolate.getNextBoneOccurence(df_position_bones, boneName, 0)
  nextBoneList = {boneName: {"frameNumber": -1, "position": lastBoneList[boneName]['position'], "rotation": lastBoneList[boneName]['rotation']} for boneName in boneList}

  newFrames = []
  for currentFrameNumber in range(0, lastFrame + 1):
    frameDf = df_position_bones[df_position_bones['frame'] == currentFrameNumber]
    presentBonesOfBoneList = [boneName for boneName in boneList if boneName in frameDf['name'].values]
    for boneName in presentBonesOfBoneList:
      if currentFrameNumber > nextBoneList[boneName]['frameNumber']:
        lastBoneList, nextBoneList = interpolate.updateBoneListsForNextOccurenceOfBone(boneName, currentFrameNumber, lastFrame, lastBoneList, nextBoneList, df_position_bones)
      if boneName == centerBone:
        for footIKBone in [footIKBone for footIKBone in footIKBones if footIKBone not in presentBonesOfBoneList]:
          interpolatedFootBonePosition = interpolate.calculateVector3Porportion(currentFrameNumber, lastBoneList[footIKBone], nextBoneList[footIKBone])
          newFrames.append([
            currentFrameNumber,
            footIKBone,
            interpolatedFootBonePosition - nextBoneList[centerBone]['position'],
            interpolate.calculateQuaternionPorportion(currentFrameNumber, lastBoneList[footIKBone], nextBoneList[footIKBone])
          ])
        for footIKBone in [footIKBone for footIKBone in footIKBones if footIKBone in presentBonesOfBoneList]:
          df_position_bones.loc[(df_position_bones['frame'] == currentFrameNumber) & (df_position_bones['name'] == footIKBone), 'position'] -= nextBoneList[centerBone]['position']
        break
      else:
        interpolatedCenterBonePosition = interpolate.calculateVector3Porportion(currentFrameNumber, lastBoneList[centerBone], nextBoneList[centerBone])
        df_position_bones.loc[(df_position_bones['frame'] == currentFrameNumber) & (df_position_bones['name'] == boneName), 'position'] -= interpolatedCenterBonePosition
  newFramesDf = pd.DataFrame(newFrames, columns=['frame', 'name', 'position', 'rotation'])
  normalized_to_center_df = df.copy()
  normalized_to_center_df.update(df_position_bones, overwrite=True)
  normalized_to_center_df = pd.concat([normalized_to_center_df, newFramesDf])
  normalized_to_center_df = df_ops.sortDf(normalized_to_center_df)
  normalized_to_center_df['complement'] = complement
  def set_position(row, centerBone):
    if row['name'] == centerBone:
      return pymeshio.common.Vector3(0, 0, 0)
    else:
      return row['position']
  normalized_to_center_df['position'] = normalized_to_center_df.apply(lambda row: set_position(row, centerBone), axis=1)
  normalized_to_center_df = df_ops.sortDf(normalized_to_center_df)
  return normalized_to_center_df

df = df_ops.loadDfFromFeather('play\\play.feather')
df = vmd_ops.filterDataframeForCoreBoneNames(df)
normalized_to_center_df = normalizePositionsToCenter(df)
interpolated_centered_df = interpolate.addInterpolationsToEachFrame(normalized_to_center_df)

In [13]:
df_ops.saveDfToFeather(interpolated_centered_df, 'play\\play_interpolated_centered.feather')

In [14]:
vmd_ops.saveDfToVmdFile(interpolated_centered_df, 'play\\play_interpolated_centered.vmd')

112200


In [ ]:
class FrameForVectorization:
  def __init__(self, df):
    self.df = df
    self.core_bone_names = self.getCoreBoneNames()

    for bone_name, attributes in self.core_bone_names.items():
      position = self.df[self.df['name'] == bone_name]['position']
      rotation = self.df[self.df['name'] == bone_name]['rotation']
      setattr(self, bone_name, {
        'pos': {
          'x': position.x,
          'y': position.y,
          'z': position.z
        },
        'hasPosition': True if 'position_weight' in attributes else False,
        'positionWeight': attributes['position_weight'] if 'position_weight' in attributes else 0,
        'rot': {
          'x': rotation.x,
          'y': rotation.y,
          'z': rotation.z,
          'w': rotation.w
        },
        'rotationWeight': attributes['rotation_weight']
      })

    @staticmethod
    def getCoreBoneNames():
      return {
        "センター": {'rotation_weight': 1, 'position_weight': 1},
        "上半身": {'rotation_weight': 1},
        "上半身2": {'rotation_weight': 1},
        "首": {'rotation_weight': 1},
        "頭": {'rotation_weight': 1},
        "下半身": {'rotation_weight': 1},
        "左肩": {'rotation_weight': 1},
        "左腕": {'rotation_weight': 1},
        "左ひじ": {'rotation_weight': 1},
        "左足": {'rotation_weight': 1},
        "左ひざ": {'rotation_weight': 1},
        "左足首": {'rotation_weight': 1},
        "左足ＩＫ": {'rotation_weight': 1, 'position_weight': 1},
        "右足ＩＫ": {'rotation_weight': 1, 'position_weight': 1},
        "右肩": {'rotation_weight': 1},
        "右腕": {'rotation_weight': 1},
        "右ひじ": {'rotation_weight': 1},
        "右足": {'rotation_weight': 1},
        "右ひざ": {'rotation_weight': 1},
        "右足首": {'rotation_weight': 1},
        "右腕捩": {'rotation_weight': 1},
        "左腕捩": {'rotation_weight': 1}
      }
      

def vectorizeFrame(df, frameNumber, animationName):
  df_frame = df[df['frame'] == frameNumber]

  vectorId = animationName + "-" + str(frameNumber)
  vector = (vectorId,)

def upsertDfAnimation(pinecone, df, animationName):
  lastFrame = df_ops.getLastFrame(df)

  vectorsOfFrames = []
  for currentFrameNumber in range(0, lastFrame + 1):
    vectorsOfFrames.append(vectorizeFrame(df, currentFrameNumber, animationName))
    

pinecone = cloud_ops.initPinecone()
upsertDfAnimation(pinecone, interpolated_centered_df, 'iroha uta')
